In [1]:
import torch
import torch.nn as nn
from torchmetrics.classification import BinaryJaccardIndex, JaccardIndex
from torchmetrics.functional.classification import multiclass_jaccard_index
from torchmetrics.functional.segmentation import mean_iou
from torch.nn.functional import softmax, one_hot

In [2]:
X = torch.rand(size=(1,3,3,8))
Y = (torch.rand(size=(1,3,3,8)) > 0.5) * 1

In [3]:
print(X)
print(Y)

tensor([[[[0.5923, 0.8477, 0.3698, 0.2617, 0.5711, 0.6682, 0.4100, 0.3150],
          [0.3169, 0.4490, 0.1383, 0.1852, 0.4072, 0.1842, 0.4442, 0.3457],
          [0.0295, 0.9735, 0.4528, 0.2898, 0.7387, 0.6396, 0.5971, 0.7699]],

         [[0.4892, 0.7381, 0.2469, 0.3066, 0.3950, 0.7744, 0.8534, 0.7583],
          [0.6708, 0.4698, 0.6715, 0.3423, 0.6955, 0.9227, 0.7397, 0.2445],
          [0.4208, 0.6996, 0.4775, 0.5555, 0.3536, 0.4516, 0.9217, 0.5818]],

         [[0.7754, 0.2985, 0.6270, 0.9628, 0.5491, 0.7306, 0.2719, 0.4139],
          [0.4095, 0.0317, 0.3471, 0.3971, 0.8689, 0.2168, 0.4625, 0.7107],
          [0.1765, 0.1216, 0.3169, 0.0588, 0.4293, 0.9679, 0.1086, 0.9597]]]])
tensor([[[[0, 0, 1, 1, 0, 0, 0, 1],
          [0, 1, 0, 1, 1, 0, 1, 0],
          [0, 0, 0, 1, 1, 0, 0, 0]],

         [[0, 0, 0, 0, 1, 0, 1, 0],
          [0, 1, 0, 0, 1, 0, 0, 1],
          [0, 1, 1, 0, 1, 1, 0, 1]],

         [[1, 1, 1, 1, 1, 1, 0, 1],
          [0, 0, 0, 1, 1, 1, 1, 1],
          [0, 1, 

In [4]:
m1 = BinaryJaccardIndex()
m2 = JaccardIndex(task='binary', average='micro')

In [5]:
print(m1(X,Y), m2(X,Y))

tensor(0.2407) tensor(0.2407)


In [6]:
# s = 0
# for i in range(7):
#     t = m1(X[i], Y[i])
#     s += t
#     print(t)
# print(s, s/8)

In [7]:
p1 = torch.mean(X, axis = (1,2))
print(p1)

tensor([[0.4312, 0.5144, 0.4053, 0.3733, 0.5565, 0.6173, 0.5343, 0.5666]])


In [8]:
print(torch.mean(X[0]))

tensor(0.4999)


In [9]:
# print(torch.mean(X[1]))

## Torchmetrics multiclass_jaccard_index vs mean_iou

In [10]:
pred = softmax(torch.rand((4,21,224,224)), dim=1)
target = torch.randint(21, (4,224,224))
target_oh = torch.permute(one_hot(target, num_classes = 21), (0,3,1,2))
pred_oh = torch.permute(one_hot(torch.argmax(pred, dim=1), num_classes = 21), (0,3,1,2))

In [11]:
mIU = mean_iou(torch.argmax(pred, dim=1), target, num_classes=21, per_class=True)
mIU1 = mean_iou(pred_oh, target_oh, num_classes=21, per_class=True)
iou = multiclass_jaccard_index(pred, target, num_classes = 21, average = None)
iou2 = multiclass_jaccard_index(pred, target, num_classes = 21)

In [12]:
print('mIU', mIU)
print('mIU1', mIU1)
print('iou', iou)
print('iou2', iou2)

mIU tensor([[0.0276, 0.0242, 0.0250, 0.0231, 0.0272, 0.0208, 0.0233, 0.0289, 0.0251,
         0.0241, 0.0258, 0.0257, 0.0236, 0.0281, 0.0261, 0.0185, 0.0242, 0.0248,
         0.0216, 0.0222, 0.0255],
        [0.0267, 0.0216, 0.0259, 0.0272, 0.0237, 0.0247, 0.0242, 0.0245, 0.0259,
         0.0231, 0.0248, 0.0285, 0.0242, 0.0261, 0.0233, 0.0238, 0.0214, 0.0263,
         0.0227, 0.0242, 0.0261],
        [0.0243, 0.0248, 0.0233, 0.0237, 0.0267, 0.0252, 0.0235, 0.0249, 0.0254,
         0.0232, 0.0224, 0.0213, 0.0233, 0.0225, 0.0229, 0.0223, 0.0239, 0.0252,
         0.0225, 0.0277, 0.0234],
        [0.0277, 0.0241, 0.0268, 0.0222, 0.0248, 0.0251, 0.0240, 0.0230, 0.0259,
         0.0208, 0.0252, 0.0268, 0.0220, 0.0243, 0.0240, 0.0252, 0.0249, 0.0244,
         0.0219, 0.0241, 0.0245]])
mIU1 tensor([[0.0276, 0.0242, 0.0250, 0.0231, 0.0272, 0.0208, 0.0233, 0.0289, 0.0251,
         0.0241, 0.0258, 0.0257, 0.0236, 0.0281, 0.0261, 0.0185, 0.0242, 0.0248,
         0.0216, 0.0222, 0.0255],
        [0

In [13]:
torch.mean(mIU, dim=0)

tensor([0.0266, 0.0237, 0.0253, 0.0240, 0.0256, 0.0240, 0.0238, 0.0253, 0.0256,
        0.0228, 0.0246, 0.0256, 0.0233, 0.0253, 0.0241, 0.0224, 0.0236, 0.0252,
        0.0222, 0.0246, 0.0248])

In [14]:
# x = torch.zeros([21])
# print(x.shape)
# for i in range(21):
#     x[i] = torch.sum(torch.where(target == i, 1, 0))
#     print(x[i])
# print(torch.sum(x))

In [15]:
4*224*224

200704

In [16]:
nc = 5
preds = torch.randint(0, 2, (4, nc, 16, 16))  # 4 samples, 5 classes, 16x16 prediction
target = torch.randint(0, 2, (4, nc, 16, 16))  # 4 samples, 5 classes, 16x16 target
print(mean_iou(preds, target, num_classes=nc))

print(mean_iou(preds, target, num_classes=nc, per_class=True))

tensor([0.3678, 0.3035, 0.3121, 0.3178])
tensor([[0.3613, 0.3679, 0.3583, 0.3909, 0.3608],
        [0.3280, 0.3622, 0.3041, 0.2732, 0.2500],
        [0.3553, 0.3316, 0.2798, 0.3065, 0.2872],
        [0.3523, 0.2745, 0.3520, 0.3503, 0.2602]])


In [17]:
print(mean_iou(torch.argmax(preds, dim=1), torch.argmax(target, dim=1), num_classes=nc))

print(mean_iou(torch.argmax(preds, dim=1), torch.argmax(target, dim=1), num_classes=nc, per_class=True))

tensor([0.1264, 0.1119, 0.1385, 0.1219])
tensor([[0.3824, 0.1239, 0.0926, 0.0333, 0.0000],
        [0.3450, 0.1083, 0.0536, 0.0000, 0.0526],
        [0.4350, 0.1635, 0.0556, 0.0385, 0.0000],
        [0.3632, 0.1545, 0.0345, 0.0571, 0.0000]])


In [18]:
multiclass_jaccard_index(preds, target, num_classes = 5, average = None)

tensor([0.3315, 0.3251, 0.0000, 0.0000, 0.0000])

In [19]:
x = list(range(2,4))

In [20]:
pred = torch.randint(5, (4,64,64))
target = torch.randint(5, (4,64,64))
target_oh = torch.permute(one_hot(target, num_classes = 5), (0,3,1,2))
pred_oh = torch.permute(one_hot(pred, num_classes = 5), (0,3,1,2))

In [21]:
mIU = mean_iou(pred, target, num_classes=5, per_class=True)
mIU1 = mean_iou(pred_oh, target_oh, num_classes=5, per_class=True)
iou = multiclass_jaccard_index(pred, target, num_classes = 5, average = None)
iou2 = multiclass_jaccard_index(pred, target, num_classes = 5)

In [22]:
# print('mIU', mIU)
print('Mean mIU', torch.mean(mIU, dim=0))
# print('mIU1', mIU1)
print('Mean mIU1', torch.mean(mIU1, dim=0))
print('iou', iou)
print('iou2', iou2)

Mean mIU tensor([0.1133, 0.1061, 0.1139, 0.1153, 0.1172])
Mean mIU1 tensor([0.1133, 0.1061, 0.1139, 0.1153, 0.1172])
iou tensor([0.1133, 0.1061, 0.1139, 0.1153, 0.1173])
iou2 tensor(0.1132)


In [23]:
torch.mean(mIU1, dim=0) / iou

tensor([1.0001, 0.9997, 0.9999, 1.0006, 0.9992])

In [24]:
z = (72 * torch.rand(21)) / torch.randint(1, 72, (21,))
z1 = (z <= 1.0)*1

In [25]:
print(z)
print(z1)

tensor([ 0.8406,  0.6046,  0.9204,  1.6259,  1.1382,  0.3609,  3.4553,  1.0171,
         0.2268,  6.5825,  4.6105,  0.4738,  5.6463, 32.3456,  1.3082,  0.3866,
         0.3696,  0.8262,  3.3581,  0.9612,  2.0770])
tensor([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0])
